In [1]:
from __future__ import print_function
import tarfile

from os import listdir
from os import remove
from os.path import exists, join, basename

from six.moves import urllib
from torchvision.transforms import Compose, CenterCrop, ToTensor, Resize
from PIL import Image

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader
import torch.utils.data as data

from math import log10
import sys
import time
import numpy as np
import argparse
import easydict

#  Prepare Data                                 <br>
Download data if don't find in local directory           <br>
Get training set from  './dataset/HR_img_train', get testset from  './dataset/HR_img_test'     <br>


In [2]:
def download_bsd300(dest="./dataset"):
    output_image_dir = join(dest, "BSDS300/images")

    if not exists(output_image_dir):
        url = "http://www2.eecs.berkeley.edu/Research/Projects/CS/vision/bsds/BSDS300-images.tgz"
        print("downloading url ", url)

        data = urllib.request.urlopen(url)

        file_path = join(dest, basename(url))
        with open(file_path, 'wb') as file:
            print("Extracting data")
        with tarfile.open(file_path) as tar:
            for item in tar:
                tar.extract(item, dest)

        remove(file_path)

    return output_image_dir


def calculate_valid_crop_size(crop_size, upscale_factor):
    return crop_size - (crop_size % upscale_factor)

def is_image_file(filename):
    return any(filename.endswith(extension) for extension in [".png", ".jpg", ".jpeg"])


def load_img(filepath):
    img = Image.open(filepath).convert('YCbCr')
    y, _, _ = img.split()
    return y


class DatasetFromFolder(data.Dataset):
    def __init__(self, image_dir, input_transform=None, target_transform=None):
        super(DatasetFromFolder, self).__init__()
        self.image_filenames = [join(image_dir, x) for x in listdir(image_dir) if is_image_file(x)]

        self.input_transform = input_transform
        self.target_transform = target_transform

    def __getitem__(self, index):
        input_image = load_img(self.image_filenames[index])     #full image
        target = input_image.copy()                             # full image
        if self.input_transform:
            input_image = self.input_transform(input_image)
        if self.target_transform:
            target = self.target_transform(target)

        return input_image, target

    def __len__(self):
        return len(self.image_filenames)


def input_transform(crop_size, upscale_factor):   #get the low resolution input by original image
    return Compose([
        CenterCrop(crop_size),                        #get a small size of data, Corp with less pieces and resize
        Resize(crop_size // upscale_factor),
        ToTensor(),
    ])


def target_transform(crop_size):
    return Compose([
        CenterCrop(crop_size),                    #get a small size of data
        ToTensor(),
    ])


def get_training_set(upscale_factor):
    root_dir = './/dataset//'
    train_dir = join(root_dir, "HR_img_train")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(train_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))


def get_test_set(upscale_factor):
    root_dir =  './/dataset//'
    test_dir = join(root_dir, "HR_img_test")
    crop_size = calculate_valid_crop_size(256, upscale_factor)

    return DatasetFromFolder(test_dir,
                             input_transform=input_transform(crop_size, upscale_factor),
                             target_transform=target_transform(crop_size))

In [64]:
a=get_training_set(4)

In [75]:
a.input_transform.transforms[0]

TypeError: 'CenterCrop' object does not support indexing

# Visualize Training precedure by design training bar and calculate training time

In [5]:
TOTAL_BAR_LENGTH = 80
LAST_T = time.time()
BEGIN_T = LAST_T


def progress_bar(current, total, msg=None):
    global LAST_T, BEGIN_T
    if current == 0:
        BEGIN_T = time.time()  # Reset for new bar.

    current_len = int(TOTAL_BAR_LENGTH * (current + 1) / total)
    rest_len = int(TOTAL_BAR_LENGTH - current_len) - 1

    sys.stdout.write(' %d/%d' % (current + 1, total))
    sys.stdout.write(' [')
    for i in range(current_len):
        sys.stdout.write('=')
    sys.stdout.write('>')
    for i in range(rest_len):
        sys.stdout.write('.')
    sys.stdout.write(']')

    current_time = time.time()
    step_time = current_time - LAST_T
    LAST_T = current_time
    total_time = current_time - BEGIN_T

    time_used = '  Step: %s' % format_time(step_time)
    time_used += ' | Tot: %s' % format_time(total_time)
    if msg:
        time_used += ' | ' + msg

    msg = time_used
    sys.stdout.write(msg)

    if current < total - 1:
        sys.stdout.write('\r')
    else:
        sys.stdout.write('\n')
    sys.stdout.flush()


# return the formatted time
def format_time(seconds):
    days = int(seconds / 3600/24)
    seconds = seconds - days*3600*24
    hours = int(seconds / 3600)
    seconds = seconds - hours*3600
    minutes = int(seconds / 60)
    seconds = seconds - minutes*60
    seconds_final = int(seconds)
    seconds = seconds - seconds_final
    millis = int(seconds*1000)

    output = ''
    time_index = 1
    if days > 0:
        output += str(days) + 'D'
        time_index += 1
    if hours > 0 and time_index <= 2:
        output += str(hours) + 'h'
        time_index += 1
    if minutes > 0 and time_index <= 2:
        output += str(minutes) + 'm'
        time_index += 1
    if seconds_final > 0 and time_index <= 2:
        output += str(seconds_final) + 's'
        time_index += 1
    if millis > 0 and time_index <= 2:
        output += str(millis) + 'ms'
        time_index += 1
    if output == '':
        output = '0ms'
    return output

#  Built CNN Model by PyTorch  

In [59]:
class Net(torch.nn.Module):
    def __init__(self, num_channels, upscale_factor, d=64, s=12, m=4):
        super(Net, self).__init__()

        self.first_part = nn.Sequential(nn.Conv2d(in_channels=num_channels, out_channels=d, kernel_size=5, stride=1, padding=2),
                                        nn.PReLU())

        self.layers = []
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=d, out_channels=s, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))
        for _ in range(m):
            self.layers.append(nn.Conv2d(in_channels=s, out_channels=s, kernel_size=3, stride=1, padding=1))
        self.layers.append(nn.PReLU())
        self.layers.append(nn.Sequential(nn.Conv2d(in_channels=s, out_channels=d, kernel_size=1, stride=1, padding=0),
                                         nn.PReLU()))

        self.mid_part = torch.nn.Sequential(*self.layers)

        self.last_part = nn.ConvTranspose2d(in_channels=d, out_channels=num_channels, kernel_size=9, stride=upscale_factor, padding=3, output_padding=1)

    def forward(self, x):
        out = self.first_part(x)
        out = self.mid_part(out)
        out = self.last_part(out)
        return out

    def weight_init(self, mean=0.0, std=0.02):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                m.weight.data.normal_(mean, std)
                if m.bias is not None:
                    m.bias.data.zero_()
            if isinstance(m, nn.ConvTranspose2d):
                m.weight.data.normal_(0.0, 0.0001)
                if m.bias is not None:
                    m.bias.data.zero_()

class FSRCNNTrainer(object):
    def __init__(self, config, training_loader, testing_loader):
        super(FSRCNNTrainer, self).__init__()
        self.CUDA = torch.cuda.is_available()
        self.device = torch.device('cuda' if self.CUDA else 'cpu')
        self.model = None
        self.lr = config.lr
        self.nEpochs = config.nEpochs
        self.criterion = None
        self.optimizer = None
        self.scheduler = None
        self.seed = config.seed
        self.upscale_factor = config.upscale_factor
        self.training_loader = training_loader
        self.testing_loader = testing_loader

    def build_model(self):
        self.model = Net(num_channels=1, upscale_factor=self.upscale_factor).to(self.device)
        self.model.weight_init(mean=0.0, std=0.2)
        self.criterion = torch.nn.MSELoss()
        torch.manual_seed(self.seed)

        if self.CUDA:
            torch.cuda.manual_seed(self.seed)
            cudnn.benchmark = True
            self.criterion.cuda()

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[50, 75, 100], gamma=0.5)  # lr decay

    def save_model(self):
        model_out_path = "model_path1.pth"
        torch.save(self.model, model_out_path)
        print("Checkpoint saved to {}".format(model_out_path))

    def train(self):
        self.model.train()
        train_loss = 0
        for batch_num, (data, target) in enumerate(self.training_loader):
            data, target = data.to(self.device), target.to(self.device)
            self.optimizer.zero_grad()
            loss = self.criterion(self.model(data), target)
            train_loss += loss.item()
            loss.backward()
            self.optimizer.step()
            progress_bar(batch_num, len(self.training_loader), 'Loss: %.4f' % (train_loss / (batch_num + 1)))

        print("    Average Loss: {:.4f}".format(train_loss / len(self.training_loader)))

    def test(self):
        self.model.eval()
        avg_psnr = 0

        with torch.no_grad():
            for batch_num, (data, target) in enumerate(self.testing_loader):
                data, target = data.to(self.device), target.to(self.device)
                prediction = self.model(data)
                mse = self.criterion(prediction, target)
                psnr = 10 * log10(1 / mse.item())
                avg_psnr += psnr
                print('')       #??
                progress_bar(batch_num, len(self.testing_loader), 'PSNR: %.4f' % (avg_psnr / (batch_num + 1)))

        print("    Average PSNR: {:.4f} dB".format(avg_psnr / len(self.testing_loader)))

    def run(self):
        self.build_model()
        for epoch in range(1, self.nEpochs + 1):
            print("\n===> Epoch {} starts:".format(epoch))
            self.train()
            self.test()
            self.scheduler.step(epoch)
            if epoch == self.nEpochs:
                self.save_model()

# Training model in 'main'

In [63]:
# ===========================================================
# Training settings
# ===========================================================
#parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
# hyper-parameters
#parser.add_argument('--batchSize', type=int, default=4, help='training batch size')
#parser.add_argument('--testBatchSize', type=int, default=1, help='testing batch size')
#parser.add_argument('--nEpochs', type=int, default=20, help='number of epochs to train for')
#parser.add_argument('--lr', type=float, default=0.01, help='Learning Rate. Default=0.01')
#parser.add_argument('--seed', type=int, default=123, help='random seed to use. Default=123')

# model configuration
#parser.add_argument('--upscale_factor', '-uf',  type=int, default=4, help="super resolution upscale factor")

args = easydict.EasyDict({
    'batchSize':4,
    'testBatchSize':1,
   'nEpochs':40,
    'lr':0.01,
    'seed':123,
    'upscale_factor':4,
    'uf':4
    
})


def main():
    # ===========================================================
    # Set train dataset & test dataset
    # ===========================================================
    print('===> Loading datasets')
    train_set = get_training_set(args.upscale_factor)
    test_set = get_test_set(args.upscale_factor)
    training_data_loader = DataLoader(dataset=train_set, batch_size=args.batchSize, shuffle=True)
    testing_data_loader = DataLoader(dataset=test_set, batch_size=args.testBatchSize, shuffle=False)

    model = FSRCNNTrainer(args, training_data_loader, testing_data_loader)

    model.run()


if __name__ == '__main__':
    main()

===> Loading datasets

===> Epoch 1 starts:
 126/126 [================================================================================>]  Step: 202ms | Tot: 1m34s | Loss: 0.9298961
    Average Loss: 0.9298

 100/100 [================================================================================>]  Step: 113ms | Tot: 18s309ms | PSNR: 22.8392
    Average PSNR: 22.8392 dB

===> Epoch 2 starts:
 126/126 [================================================================================>]  Step: 159ms | Tot: 1m29s | Loss: 0.00404
    Average Loss: 0.0040

 18/100 [==============>.................................................................]  Step: 209ms | Tot: 2s866ms | PSNR: 25.3119


 100/100 [================================================================================>]  Step: 112ms | Tot: 17s729ms | PSNR: 25.3599
    Average PSNR: 25.3599 dB

===> Epoch 3 starts:
 126/126 [================================================================================>]  Step: 161ms | Tot: 1m28s | Loss: 0.0027828
    Average Loss: 0.0027

 37/100 [=============================>..................................................]  Step: 217ms | Tot: 6s160ms | PSNR: 26.1973


 100/100 [================================================================================>]  Step: 117ms | Tot: 17s378ms | PSNR: 25.8296
    Average PSNR: 25.8296 dB

===> Epoch 4 starts:
 126/126 [================================================================================>]  Step: 236ms | Tot: 1m28s | Loss: 0.00213
    Average Loss: 0.0021

 56/100 [============================================>...................................]  Step: 267ms | Tot: 9s638ms | PSNR: 27.1162


 100/100 [================================================================================>]  Step: 113ms | Tot: 17s110ms | PSNR: 26.9927
    Average PSNR: 26.9927 dB

===> Epoch 5 starts:
 126/126 [================================================================================>]  Step: 149ms | Tot: 1m27s | Loss: 0.001990
    Average Loss: 0.0019

 75/100 [============================================================>...................]  Step: 181ms | Tot: 12s533ms | PSNR: 27.9525


 100/100 [================================================================================>]  Step: 109ms | Tot: 16s670ms | PSNR: 28.2016
    Average PSNR: 28.2016 dB

===> Epoch 6 starts:
 126/126 [================================================================================>]  Step: 315ms | Tot: 1m27s | Loss: 0.0018920
    Average Loss: 0.0018

 94/100 [===========================================================================>....]  Step: 223ms | Tot: 15s752ms | PSNR: 28.7802


...

 100/100 [================================================================================>]  Step: 121ms | Tot: 18s490ms | PSNR: 32.5083
    Average PSNR: 32.5083 dB

===> Epoch 27 starts:
 126/126 [===============================================================================>]  Step: 902ms | Tot: 1m34s | Loss: 0.00076006===================================>]  Step: 172ms | Tot: 1m34s | Loss: 0.0008
    Average Loss: 0.0008

 66/100 [====================================================>...........................]  Step: 178ms | Tot: 11s798ms | PSNR: 30.0419


 100/100 [================================================================================>]  Step: 126ms | Tot: 18s266ms | PSNR: 30.3527
    Average PSNR: 30.3527 dB

===> Epoch 28 starts:
 126/126 [===============================================================================>]  Step: 653ms | Tot: 1m34s | Loss: 0.00070708 [================================================================================>]  Step: 183ms | Tot: 1m34s | Loss: 0.0007
    Average Loss: 0.0007

 84/100 [===================================================================>............]  Step: 172ms | Tot: 14s613ms | PSNR: 32.9602


 100/100 [================================================================================>]  Step: 106ms | Tot: 17s302ms | PSNR: 32.9934
    Average PSNR: 32.9934 dB

===> Epoch 29 starts:
 126/126 [===============================================================================>]  Step: 631ms | Tot: 1m31s | Loss: 0.00133666 [================================================================================>]  Step: 185ms | Tot: 1m31s | Loss: 0.0015
    Average Loss: 0.0015

 100/100 [================================================================================>]  Step: 115ms | Tot: 17s909ms | PSNR: 21.0933
    Average PSNR: 21.0933 dB

===> Epoch 30 starts:
 126/126 [================================================================================>]  Step: 239ms | Tot: 1m34s | Loss: 53410980.75804645
    Average Loss: 53410980.7580



 100/100 [================================================================================>]  Step: 106ms | Tot: 17s698ms | PSNR: -46.5043
    Average PSNR: -46.5043 dB

===> Epoch 31 starts:


KeyboardInterrupt: 

# Output high resolution image

In [77]:
# ===========================================================
# Argument settings
# ===========================================================
#parser = argparse.ArgumentParser(description='PyTorch Super Res Example')
#parser.add_argument('--input', type=str, required=False, default='./dataset/', help='input image to use')
#parser.add_argument('--model', type=str, default='model_path.pth', help='model file to use')
#parser.add_argument('--output', type=str, default='test.jpg', help='where to save the output image')
#args = parser.parse_args()
#print(args)

test = easydict.EasyDict({
    'input':'.//test_image//0.jpg',
    'model':'model_path.pth',
   'output':'.//test_image//0_generate.jpg',

    
})

# ===========================================================
# input image setting
# ===========================================================
GPU_IN_USE = torch.cuda.is_available()
img = Image.open(test.input).convert('YCbCr')
y, cb, cr = img.split()


# ===========================================================
# model import & setting
# ===========================================================
device = torch.device('cuda' if GPU_IN_USE else 'cpu')
model = torch.load(test.model, map_location=lambda storage, loc: storage)
model = model.to(device)
data = (ToTensor()(y)).view(1, -1, y.size[1], y.size[0])
data = data.to(device)

if GPU_IN_USE:
    cudnn.benchmark = True


# ===========================================================
# output and save image
# ===========================================================
out = model(data)
out = out.cpu()
out_img_y = out.data[0].numpy()
out_img_y *= 255.0
out_img_y = out_img_y.clip(0, 255)
out_img_y = Image.fromarray(np.uint8(out_img_y[0]), mode='L')

out_img_cb = cb.resize(out_img_y.size, Image.BICUBIC)
out_img_cr = cr.resize(out_img_y.size, Image.BICUBIC)
out_img = Image.merge('YCbCr', [out_img_y, out_img_cb, out_img_cr]).convert('RGB')

out_img.save(test.output)
print('output image saved to ', test.output)

output image saved to  .//test_image//0_generate.jpg
